In [1]:
# Import libraries
import pandas as pd
import numpy as np

from sklearn.model_selection import KFold
from sklearn.metrics import classification_report, accuracy_score, f1_score

from imblearn.over_sampling import SMOTE

from sklearn.ensemble import (
    AdaBoostClassifier,
    BaggingClassifier,
    ExtraTreesClassifier,
    GradientBoostingClassifier,
    HistGradientBoostingClassifier,
    RandomForestClassifier
)

In [2]:
# Load the processed dataset
df = pd.read_csv('../data/preprocessed/with_diabetes_status/dataset_with_diabetes_status.csv')

In [3]:
# Drop the diabetes and hba1c columns
df = df.drop('diabetes', axis=1)

In [4]:
# Drop records where gender is 'Other'
df = df[df['gender'] != 'Other']

In [5]:
# One-hot encode categorical features
df = pd.get_dummies(df, columns=['gender', 'smoking_history'], drop_first=False)

In [6]:
# Define the order of categories for diabetes_status
status_order = ['non diabetic', 'stress induced prediabetic', 'stress induced type 2 diabetic', 'prediabetic', 'diabetic']

# Create a mapping for the specified order
status_mapping = {status: i for i, status in enumerate(status_order)}

# Map 'diabetes_status' to the numeric encoding
df['diabetes_status'] = df['diabetes_status'].map(status_mapping)

In [7]:
# Move the encoded 'diabetes_status' to the right
cols = df.columns.tolist()
cols.append(cols.pop(cols.index('diabetes_status')))
df = df[cols]

In [8]:
# Separate features and target
X = df.drop(columns=['diabetes_status'])
y = df['diabetes_status']

In [9]:
# Initialize SMOTE
smote = SMOTE(random_state=42)
X_res, y_res = smote.fit_resample(X, y)

In [10]:
# Initialize KFold
kf = KFold(n_splits=5, shuffle=True, random_state=42)

In [11]:
# List of models to evaluate
models = [
    AdaBoostClassifier(),
    BaggingClassifier(),
    ExtraTreesClassifier(),
    GradientBoostingClassifier(),
    HistGradientBoostingClassifier(),
    RandomForestClassifier()
]

In [12]:
def evaluate_models(models, X, y, kf):
    results = {}
    
    for model in models:
        model_name = type(model).__name__
        print(f"Evaluating {model_name}...")
        
        fold_accuracies = []
        fold_f1_scores = []
        fold_reports = []
        
        for fold, (train_index, test_index) in enumerate(kf.split(X)):
            X_train, X_test = X.iloc[train_index], X.iloc[test_index]
            y_train, y_test = y.iloc[train_index], y.iloc[test_index]
            
            model.fit(X_train, y_train)
            y_pred = model.predict(X_test)
            
            accuracy = accuracy_score(y_test, y_pred)
            f1 = f1_score(y_test, y_pred, average='weighted')
            report = classification_report(y_test, y_pred)
            
            fold_accuracies.append(accuracy)
            fold_f1_scores.append(f1)
            fold_reports.append(report)
            
            print(f"Fold {fold + 1} - Accuracy: {accuracy:.4f}, F1 Score: {f1:.4f}")
            print(f"Classification Report for Fold {fold + 1}:\n{report}\n")
        
        mean_accuracy = np.mean(fold_accuracies)
        std_accuracy = np.std(fold_accuracies)
        mean_f1 = np.mean(fold_f1_scores)
        std_f1 = np.std(fold_f1_scores)
        
        print(f"\n{model_name} - Mean Accuracy: {mean_accuracy:.4f} (± {std_accuracy:.4f}), Mean F1 Score: {mean_f1:.4f} (± {std_f1:.4f})\n")
        
        results[model_name] = {
            'mean_accuracy': mean_accuracy,
            'std_accuracy': std_accuracy,
            'mean_f1': mean_f1,
            'std_f1': std_f1,
            'reports': fold_reports
        }
    
    return results

In [13]:
# Call the function with the updated list of models, features, target, and KFold object
results = evaluate_models(models, X_res, y_res, kf)

Evaluating AdaBoostClassifier...


C:\Users\david\anaconda3\Lib\site-packages\sklearn\ensemble\_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
C:\Users\david\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\david\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\david\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Preci

Fold 1 - Accuracy: 0.5935, F1 Score: 0.4594
Classification Report for Fold 1:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      7583
           1       0.00      0.00      0.00      7703
           2       0.49      1.00      0.66      7362
           3       0.50      1.00      0.66      7626
           4       0.00      0.00      0.00      7755

    accuracy                           0.59     38029
   macro avg       0.40      0.60      0.46     38029
weighted avg       0.39      0.59      0.46     38029




C:\Users\david\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\david\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\david\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\david\anaconda3\Lib\site-packag

Fold 2 - Accuracy: 0.5974, F1 Score: 0.4640
Classification Report for Fold 2:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      7650
           1       0.50      1.00      0.66      7541
           2       0.00      0.00      0.00      7678
           3       0.00      0.00      0.00      7631
           4       0.50      1.00      0.66      7529

    accuracy                           0.60     38029
   macro avg       0.40      0.60      0.47     38029
weighted avg       0.40      0.60      0.46     38029




...

Fold 3 - Accuracy: 0.5977, F1 Score: 0.4644
Classification Report for Fold 3:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      7683
           1       0.00      0.00      0.00      7585
           2       0.00      0.00      0.00      7714
           3       0.50      1.00      0.67      7544
           4       0.49      1.00      0.66      7503

    accuracy                           0.60     38029
   macro avg       0.40      0.60      0.47     38029
weighted avg       0.40      0.60      0.46     38029




C:\Users\david\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\david\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\david\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\david\anaconda3\Lib\site-packag

Fold 4 - Accuracy: 0.5967, F1 Score: 0.4626
Classification Report for Fold 4:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      7479
           1       0.00      0.00      0.00      7676
           2       0.00      0.00      0.00      7662
           3       0.50      1.00      0.66      7556
           4       0.50      1.00      0.67      7656

    accuracy                           0.60     38029
   macro avg       0.40      0.60      0.47     38029
weighted avg       0.40      0.60      0.46     38029




C:\Users\david\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\david\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\david\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Fold 5 - Accuracy: 0.5981, F1 Score: 0.4646
Classification Report for Fold 5:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      7634
           1       0.50      1.00      0.66      7524
           2       0.00      0.00      0.00      7613
           3       0.00      0.00      0.00      7672
           4       0.50      1.00      0.67      7586

    accuracy                           0.60     38029
   macro avg       0.40      0.60      0.47     38029
weighted avg       0.40      0.60      0.46     38029



AdaBoostClassifier - Mean Accuracy: 0.5967 (± 0.0016), Mean F1 Score: 0.4630 (± 0.0019)

Evaluating BaggingClassifier...
Fold 1 - Accuracy: 0.9776, F1 Score: 0.9776
Classification Report for Fold 1:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      7583
           1       0.95      0.98      0.97      7703
           2       0.96      0.99      0.98      7362
           3   